# Import libraries

In [1]:
import requests
#BeautifulSoup
from bs4 import BeautifulSoup as bs 

#Selenium
try:
  from selenium import webdriver
  from selenium.webdriver.common.keys import Keys
  from webdriver_manager.chrome import ChromeDriverManager
  from selenium.webdriver.support.select import Select #Select DropDown Menu
  from selenium.webdriver.support import expected_conditions as EC

except:
  !pip install selenium #Selenium 설치
  !apt-get update
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install webdriver-manager
  from selenium import webdriver
  from selenium.webdriver.common.keys import Keys
  from webdriver_manager.chrome import ChromeDriverManager
  from selenium.webdriver.support.select import Select #Select DropDown Menu
  from selenium.webdriver.support import expected_conditions as EC

import os
import time
from tqdm import tqdm #showing process bar
import pandas as pd
import csv
import re
import numpy as np

In [2]:
#access Google drive file
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Web Crawling

## KCGS (한국기업지배구조) ESG 등급 데이터 가져오기
- [2021 ~ 2019 ESG등급](https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp)

In [7]:
def get_ESG_new(driver, year, path):
  data = []
  elements = []
  pagination = True
  page = 1
  pg = 1
  url = 'https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp'

  #url 접속
  driver.get(url)
  year_option = {2022 : 1, 2021:2, 2020:3, 2019:4}
  index = year_option.get(year)
  option = driver.find_element_by_xpath(f'/html/body/div/div[3]/div[2]/div[2]/article/fieldset/form/div/div[1]/dl/dd[2]/select/option[{index}]') #연도 선택
  driver.execute_script("return arguments[0].selected=true;", option)
  time.sleep(5)
  driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/div[2]/article/fieldset/form/div/div[2]/button').click() #조회 버튼 클릭
  time.sleep(5)

  try:
   while(pagination):
      xpath ='/html/body/div/div[3]/div[2]/div[2]/article/div[5]/table/tbody'
      table = driver.find_element_by_xpath(xpath)
      for tr in table.find_elements_by_tag_name('tr'):
        td = tr.find_elements_by_tag_name('td')
        #MSCI 제외하고, KCGS - ESG 등급만 가져오기
        for i in range(5):
          if i == 0:
            elements.append(td[i].find_element_by_tag_name('a').text) #기업명 가져오기
          else:
            class_name = td[i].find_element_by_tag_name('span').get_attribute('class') #아이콘이라서 class name 비교해 등급값 가져오기
            if ('grd-icon kcgs-icon kcgs1' == class_name):
              elements.append('D')
            elif ('grd-icon kcgs-icon kcgs2' == class_name):
              elements.append('C')
            elif ('grd-icon kcgs-icon kcgs3' == class_name):
              elements.append('B')
            elif ('grd-icon kcgs-icon kcgs4' == class_name):
              elements.append('B+')
            elif ("grd-icon kcgs-icon kcgs5" == class_name):
              elements.append('A')
            elif ('grd-icon kcgs-icon kcgs6' == class_name):
              elements.append('A+')
            else:
              elements.append('-')

        elements.append(year) #연도값 추가하기
        data.append(elements)
        elements = []
    
       
      # 한 페이지 다 크롤링하면 다음 페이지로 이동
      try:
        if (page % 10 == 0):
          page = 1
          pg += 1
          driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/div[2]/article/div[5]/div[2]/ul/li[13]/a').click()
          driver.implicitly_wait(3)
          time.sleep(8)
          print(f'\n {pg}페이지 이동 ', end='')
      
        else:
          page += 1
          pg += 1
          driver.find_element_by_xpath(f'/html/body/div/div[3]/div[2]/div[2]/article/div[5]/div[2]/ul/li[{page+2}]/a').click() 
          driver.implicitly_wait(3)
          time.sleep(8)
          print(f'{pg}페이지 이동 ', end = '')
      except:
        pagination = False
        print('마지막 페이지입니다.')
  
   columns = ['기업명', ' ESG 종합등급', '환경', '사회', '지배구조', '평가연도']
   df = pd.DataFrame(columns=columns, data=data)
   df.to_csv(path + f'KCGS_ESG등급_{year}.csv')
   print('크롤링 완료')

  except Exception as error:
    print('오류 발생')
    print(error)
   
  finally:
    #driver 종료
    driver.quit()


## KCGS 신용평가 ESG 등급 데이터 가져오기
- [2018 ~ 2011 ESG등급](http://data.krx.co.kr/contents/MDC/HARD/hardController/MDCHARD050.cmd#none)

In [4]:
def get_ESG_old(driver, year, path):
  data = []
  elements = []
  pagination = True
  page = 1
  pg = 1
  url = 'http://data.krx.co.kr/contents/MDC/HARD/hardController/MDCHARD050.cmd'

  #url 접속
  driver.get(url)
  year_option = {2018 : 1, 2017:2, 2016:3, 2015:4, 2014:5, 2013:6, 2012:7, 2011:8}
  index = year_option.get(year)

  option = driver.find_element_by_xpath(f'//*[@id="grdYy"]/option[{index}]')
  driver.execute_script("return arguments[0].selected=true;", option) #연도 선택
  time.sleep(3)
  driver.find_element_by_xpath('//*[@id="jsSearchButton"]').click() #조회 버튼 클릭
  time.sleep(3)
  
  try:
    xpath = '//*[@id="jsTable_MDCHARD050_0"]/tbody'
    table = driver.find_element_by_xpath(xpath)
    for tr in tqdm(table.find_elements_by_tag_name('tr')):
      td = tr.find_elements_by_tag_name('td')
      for i in range(6):
        try:
          elements.append(td[i].text)
        except:
          elements.append('-') #비어 있는 값

      data.append(elements)
      elements = []
    
       
    columns = ['기업명', ' ESG 종합등급', '지배구조', '사회', '환경', '평가연도']
    df = pd.DataFrame(columns=columns, data=data)
    df.to_csv(path + f'KCGS_ESG등급_{year}.csv')
    print('크롤링 완료')

  except Exception as error:
    print('오류 발생')
    print(error)
   
  finally:
    #driver 종료
    driver.quit()


In [8]:
options = webdriver.ChromeOptions()

#set options
options.add_argument('headless') #브라우저 창 안열고, 크롤링
#headless 옵션 추가하지 않으면, Chrome failed to start 에러 발생할 수 있음
options.add_argument('--no-sandbox') #보안 기능인 sandbox 비활성화
options.add_argument('--disable-dev-shm-usage') # 공유 메모리 /dev/shm 디렉토리 미사용
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ['enable-automation'])

driver = webdriver.Chrome('chromedriver', options=options)
#완전히 로드될 때까지 최대 7초 기다림 
#time.sleep()<프로세스 자체를 지정한 시간동안 기다림>과 비슷한데, 더 빠름
driver.implicitly_wait(5) #브라우저에서 사용되는 엔진 자체에서 파싱되는 시간을 기다림

In [21]:
path = '/content/drive/MyDrive/LIS3821/datasets/'
get_ESG_new(driver, 2021, path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/i

2페이지 이동 3페이지 이동 4페이지 이동 5페이지 이동 6페이지 이동 7페이지 이동 8페이지 이동 9페이지 이동 10페이지 이동 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead



 11페이지 이동 12페이지 이동 13페이지 이동 14페이지 이동 15페이지 이동 16페이지 이동 17페이지 이동 18페이지 이동 19페이지 이동 20페이지 이동 
 21페이지 이동 22페이지 이동 23페이지 이동 24페이지 이동 25페이지 이동 26페이지 이동 27페이지 이동 28페이지 이동 29페이지 이동 30페이지 이동 
 31페이지 이동 32페이지 이동 33페이지 이동 34페이지 이동 35페이지 이동 36페이지 이동 37페이지 이동 38페이지 이동 39페이지 이동 40페이지 이동 
 41페이지 이동 42페이지 이동 43페이지 이동 44페이지 이동 45페이지 이동 46페이지 이동 47페이지 이동 48페이지 이동 49페이지 이동 50페이지 이동 
 51페이지 이동 52페이지 이동 53페이지 이동 54페이지 이동 55페이지 이동 56페이지 이동 57페이지 이동 58페이지 이동 59페이지 이동 60페이지 이동 
 61페이지 이동 62페이지 이동 63페이지 이동 64페이지 이동 65페이지 이동 66페이지 이동 67페이지 이동 68페이지 이동 69페이지 이동 70페이지 이동 
 71페이지 이동 72페이지 이동 73페이지 이동 74페이지 이동 75페이지 이동 76페이지 이동 77페이지 이동 78페이지 이동 마지막 페이지입니다.
크롤링 완료


In [23]:
path = '/content/drive/MyDrive/LIS3821/datasets/'
get_ESG_new(driver, 2020, path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/i

2페이지 이동 3페이지 이동 4페이지 이동 5페이지 이동 6페이지 이동 7페이지 이동 8페이지 이동 9페이지 이동 10페이지 이동 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead



 11페이지 이동 12페이지 이동 13페이지 이동 14페이지 이동 15페이지 이동 16페이지 이동 17페이지 이동 18페이지 이동 19페이지 이동 20페이지 이동 
 21페이지 이동 22페이지 이동 23페이지 이동 24페이지 이동 25페이지 이동 26페이지 이동 27페이지 이동 28페이지 이동 29페이지 이동 30페이지 이동 
 31페이지 이동 32페이지 이동 33페이지 이동 34페이지 이동 35페이지 이동 36페이지 이동 37페이지 이동 38페이지 이동 39페이지 이동 40페이지 이동 
 41페이지 이동 42페이지 이동 43페이지 이동 44페이지 이동 45페이지 이동 46페이지 이동 47페이지 이동 48페이지 이동 49페이지 이동 50페이지 이동 
 51페이지 이동 52페이지 이동 53페이지 이동 54페이지 이동 55페이지 이동 56페이지 이동 57페이지 이동 58페이지 이동 59페이지 이동 60페이지 이동 
 61페이지 이동 62페이지 이동 63페이지 이동 64페이지 이동 65페이지 이동 66페이지 이동 67페이지 이동 68페이지 이동 69페이지 이동 70페이지 이동 
 71페이지 이동 72페이지 이동 73페이지 이동 74페이지 이동 75페이지 이동 76페이지 이동 77페이지 이동 78페이지 이동 마지막 페이지입니다.
크롤링 완료


In [9]:
path = '/content/drive/MyDrive/LIS3821/datasets/'
get_ESG_new(driver, 2019, path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
/usr/local/lib/python3.7/dist-packages/i

2페이지 이동 3페이지 이동 4페이지 이동 5페이지 이동 6페이지 이동 7페이지 이동 8페이지 이동 9페이지 이동 10페이지 이동 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead



 11페이지 이동 12페이지 이동 13페이지 이동 14페이지 이동 15페이지 이동 16페이지 이동 17페이지 이동 18페이지 이동 19페이지 이동 20페이지 이동 
 21페이지 이동 22페이지 이동 23페이지 이동 24페이지 이동 25페이지 이동 26페이지 이동 27페이지 이동 28페이지 이동 29페이지 이동 30페이지 이동 
 31페이지 이동 32페이지 이동 33페이지 이동 34페이지 이동 35페이지 이동 36페이지 이동 37페이지 이동 38페이지 이동 39페이지 이동 40페이지 이동 
 41페이지 이동 42페이지 이동 43페이지 이동 44페이지 이동 45페이지 이동 46페이지 이동 47페이지 이동 48페이지 이동 49페이지 이동 50페이지 이동 
 51페이지 이동 52페이지 이동 53페이지 이동 54페이지 이동 55페이지 이동 56페이지 이동 57페이지 이동 58페이지 이동 59페이지 이동 60페이지 이동 
 61페이지 이동 62페이지 이동 63페이지 이동 64페이지 이동 65페이지 이동 66페이지 이동 67페이지 이동 68페이지 이동 69페이지 이동 70페이지 이동 
 71페이지 이동 72페이지 이동 73페이지 이동 74페이지 이동 75페이지 이동 76페이지 이동 77페이지 이동 78페이지 이동 마지막 페이지입니다.
크롤링 완료


In [11]:
get_ESG_old(driver, 2018, path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
100%|██████████| 747/747 [02:35<00:00,  4.81it/s]

크롤링 완료


In [14]:
get_ESG_old(driver, 2017, path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
100%|██████████| 733/733 [02:36<00:00,  4.68it/s]

크롤링 완료
